In [1]:
import os
import keras
import numpy as np
import tensorflow as tf
import neuralspot_edge as nse
import matplotlib.pyplot as plt

2024-08-20 21:33:27.071126: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-20 21:33:27.077083: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-20 21:33:27.084959: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-20 21:33:27.087290: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-20 21:33:27.093566: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Constants
AUG_SIZE = 64
CROP_SIZE = 64
NUM_CLASSES = 10
SHUFFLE_SIZE = 10000
BATCH_SIZE = 256
EPOCHS = 100
LR_RATE = 1e-3
LR_CYCLES = 1
VERBOSE = 1

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
y_train = y_train.squeeze()
y_test = y_test.squeeze()

In [4]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
val_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))

I0000 00:00:1724189608.811277   13335 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1724189608.829588   13335 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1724189608.829684   13335 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1724189608.830879   13335 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [5]:
# # Plot a random image
# fig, ax = plt.subplots(figsize=(2, 2))
# ax.imshow(x_train[np.random.randint(0, x_train.shape[0])])
# _ = ax.axis('off')

In [6]:
augmenter = nse.layers.preprocessing.AugmentationPipeline(
    layers=[
        nse.layers.preprocessing.Resizing2D(
            height=AUG_SIZE,
            width=AUG_SIZE,
            name="Resizing"
        ),
        # nse.layers.preprocessing.RandomFlip2D(
        #     horizontal=True,
        #     vertical=False,
        #     name="RandomFlip"
        # ),
        # nse.layers.preprocessing.RandomCutout2D(
        #     factor=(0.01, 0.05),
        #     cutouts=2,
        #     fill_mode="constant",
        #     fill_value=0.0,
        #     name="RandomCutout"
        # ),
        # nse.layers.preprocessing.RandomCrop2D(
        #     height=CROP_SIZE,
        #     width=CROP_SIZE,
        #     name="RandomCrop",
        #     auto_vectorize=True
        # ),
        # nse.layers.preprocessing.Rescaling2D(
        #     scale=1.0 / 255.0,
        #     name="Rescaling"
        # )
    ],
)

In [7]:
y = augmenter(x_train[:1].astype(np.float32))

In [8]:
# # Plot a random image
# fig, ax = plt.subplots(figsize=(2, 2))
# ax.imshow(y[0].numpy())
# _ = ax.axis('off')


In [9]:
train_ds = train_ds.shuffle(
    buffer_size=SHUFFLE_SIZE
).batch(
    batch_size=BATCH_SIZE
).map(
    lambda data, labels: {
        "data": tf.cast(data, "float32"),
        "labels": tf.one_hot(labels, NUM_CLASSES),
    },
    num_parallel_calls=tf.data.AUTOTUNE,
).map(
    augmenter,
    num_parallel_calls=tf.data.AUTOTUNE
).map(
    lambda data: (data["data"], data["labels"]),
    num_parallel_calls=tf.data.AUTOTUNE,
).prefetch(
    tf.data.AUTOTUNE
)

In [10]:
val_ds = val_ds.shuffle(
    buffer_size=SHUFFLE_SIZE
).batch(
    batch_size=BATCH_SIZE
).map(
    lambda data, labels: {
        "data": tf.cast(data, "float32"),
        "labels": tf.one_hot(labels, NUM_CLASSES),
    },
    num_parallel_calls=tf.data.AUTOTUNE,
).map(
    augmenter,
    num_parallel_calls=tf.data.AUTOTUNE
).map(
    lambda data: (data["data"], data["labels"]),
    num_parallel_calls=tf.data.AUTOTUNE,
).cache().prefetch(
    tf.data.AUTOTUNE
)

In [11]:
inputs = keras.Input(shape=(CROP_SIZE, CROP_SIZE, 3), name='inputs')

model = nse.models.EfficientNetV2(
    x=inputs,
    params=nse.models.EfficientNetParams(
        input_filters=32,
        input_kernel_size=(3, 3),
        input_strides=(2, 2),
        blocks=[
            nse.models.MBConvParams(filters=48, depth=2, kernel_size=(3, 3), strides=(2, 2), ex_ratio=1, se_ratio=4),
            nse.models.MBConvParams(filters=64, depth=2, kernel_size=(3, 3), strides=(2, 2), ex_ratio=1, se_ratio=4),
            nse.models.MBConvParams(filters=96, depth=2, kernel_size=(3, 3), strides=(1, 1), ex_ratio=1, se_ratio=4),
            nse.models.MBConvParams(filters=128, depth=1, kernel_size=(3, 3), strides=(2, 2), ex_ratio=1, se_ratio=4)
        ],
        output_filters=0,
        include_top=True,
        dropout=0.2,
        drop_connect_rate=0.0,
        model_name="efficientnetv2"
    ),
    num_classes=NUM_CLASSES,
)

In [12]:
flops = nse.metrics.flops.get_flops(model, batch_size=1, fpath=os.devnull)
model.summary(layer_range=('inputs', model.layers[10].name))
print(f"FLOPS: {flops/1e6:.2f}M")

Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.


[08/20/24 21:33:29] WARNING  WARNING:tensorflow:From                                              ]8;id=561795;file:///workspaces/neuralspot-edge/.venv/lib/python3.12/site-packages/tensorflow/python/util/deprecation.py\deprecation.py]8;;\:]8;id=606999;file:///workspaces/neuralspot-edge/.venv/lib/python3.12/site-packages/tensorflow/python/util/deprecation.py#50\50]8;;\
                             /workspaces/neuralspot-edge/.venv/lib/python3.12/site-packages/tenso                  
                             rflow/python/ops/nn_ops.py:5256: tensor_shape_from_node_def_name                      
                             (from tensorflow.python.framework.graph_util_impl) is deprecated and                  
                             will be removed in a future version.                                                  
                             Instructions for updating:                                                            
                             This API was designed for TensorFlow v1. See                                          
                             https://www.tensorflow.org/guide/migrate for instructions on how to                   
                             migrate your code to TensorFlow v2.                                                   

Model: "EfficientNetV2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ inputs (InputLayer) │ (None, 48, 48, 3) │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem.conv (Conv2D)  │ (None, 24, 24,    │        432 │ inputs[0][0]      │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem.bn             │ (None, 24, 24,    │         64 │ stem.conv[0][0]   │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem.act            │ (None, 24, 24,    │          0 │ stem.bn[0][0]     │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stage1.mbconv1.dp   │ (None, 12, 12,    │        144 │ stem.act[0][0]    │
│ (DepthwiseConv2D)   │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stage1.mbconv1.dp.… │ (None, 12, 12,    │         64 │ stage1.mbconv1.d… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stage1.mbconv1.dp.… │ (None, 12, 12,    │          0 │ stage1.mbconv1.d… │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stage1.mbconv1.se.… │ (None, 1, 1, 16)  │          0 │ stage1.mbconv1.d… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stage1.mbconv1.se.… │ (None, 1, 1, 4)   │         68 │ stage1.mbconv1.s… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stage1.mbconv1.se.… │ (None, 1, 1, 4)   │          0 │ stage1.mbconv1.s… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stage1.mbconv1.se.… │ (None, 1, 1, 16)  │         80 │ stage1.mbconv1.s… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stage1.mbconv1.se.… │ (None, 1, 1, 16)  │          0 │ stage1.mbconv1.s… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 12, 12,    │          0 │ stage1.mbconv1.d… │
│                     │ 16)               │            │ stage1.mbconv1.s… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stage1.mbconv1.red… │ (None, 12, 12,    │        512 │ multiply[0][0]    │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stage1.mbconv1.red… │ (None, 12, 12,    │        128 │ stage1.mbconv1.r… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stage1.mbconv2.dp   │ (None, 12, 12,    │        288 │ stage1.mbconv1.r… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stage1.mbconv2.dp.… │ (None, 12, 12,    │        128 │ stage1.mbconv2.d… │
│ (BatchNormalizatio… │ 32)               │            │                 

 Total params: 33,574 (131.15 KB)

 Trainable params: 32,166 (125.65 KB)

 Non-trainable params: 1,408 (5.50 KB)

FLOPS: 2.25M


In [13]:
t_mul = 1
first_steps = (EPOCHS * len(train_ds)) / (np.power(LR_CYCLES, t_mul) - t_mul + 1)
scheduler = keras.optimizers.schedules.CosineDecayRestarts(
    initial_learning_rate=LR_RATE,
    first_decay_steps=np.ceil(first_steps),
    t_mul=t_mul,
    m_mul=0.5,
)

optimizer = keras.optimizers.Adam(scheduler)
loss = keras.losses.CategoricalCrossentropy(from_logits=True)

metrics = [
    keras.metrics.CategoricalAccuracy(name="acc"),
    keras.metrics.F1Score(name="f1", average="weighted"),
]

callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=5,
        restore_best_weights=True,
        verbose=VERBOSE
    ),
    keras.callbacks.ModelCheckpoint(
        monitor="val_loss",
        verbose=VERBOSE,
        save_best_only=True,
        filepath="model.keras"
    )
]


In [14]:
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics,
)

In [15]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=VERBOSE,
)

Epoch 1/100


2024-08-20 21:33:29.808636: E tensorflow/core/util/util.cc:131] oneDNN supports DT_INT32 only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.
I0000 00:00:1724189613.445542   13499 service.cc:146] XLA service 0x73bb340041e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1724189613.445579   13499 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-08-20 21:33:33.606633: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-20 21:33:34.050872: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


 23/196 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - acc: 0.1240 - f1: 0.1224 - loss: 2.3915

I0000 00:00:1724189619.357328   13499 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


196/196 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - acc: 0.2184 - f1: 0.2120 - loss: 2.1092

2024-08-20 21:33:47.270602: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_726', 4 bytes spill stores, 4 bytes spill loads




Epoch 1: val_loss improved from inf to 1.75199, saving model to model.keras
196/196 ━━━━━━━━━━━━━━━━━━━━ 19s 50ms/step - acc: 0.2187 - f1: 0.2123 - loss: 2.1083 - val_acc: 0.3396 - val_f1: 0.3227 - val_loss: 1.7520
Epoch 2/100
193/196 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - acc: 0.3617 - f1: 0.3554 - loss: 1.6896
Epoch 2: val_loss improved from 1.75199 to 1.61905, saving model to model.keras
196/196 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - acc: 0.3620 - f1: 0.3556 - loss: 1.6890 - val_acc: 0.3973 - val_f1: 0.3899 - val_loss: 1.6190
Epoch 3/100
194/196 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - acc: 0.3992 - f1: 0.3939 - loss: 1.5985
Epoch 3: val_loss improved from 1.61905 to 1.55375, saving model to model.keras
196/196 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - acc: 0.3994 - f1: 0.3941 - loss: 1.5980 - val_acc: 0.4278 - val_f1: 0.4199 - val_loss: 1.5538
Epoch 4/100
196/196 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - acc: 0.4441 - f1: 0.4397 - loss: 1.5015
Epoch 4: val_loss improved from 1.55375 to 1.50824, saving model